In [1]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_memory import ChatMessageHistory
from langchain.memory.chat_message_histories import RedisChatMessageHistory
from langchain import OpenAI, LLMChain
from langchain.chat_models import ChatOpenAI

import configparser, os
config = configparser.ConfigParser()
config.read('./keys.ini')
openai_api_key = config['OPENAI']['OPENAI_API_KEY']

from agent_tools import tools
from agents import *

In [2]:
message_history = RedisChatMessageHistory(url='redis://localhost:6379/0', ttl=600, session_id='buffer')
memory = ConversationBufferMemory(memory_key="chat_history", chat_memory=message_history)

prefix = """You are PRTS / 普瑞赛斯, an AI assistant to answer questions about the Arknigths story by querying vector database. 
"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "chat_history", "agent_scratchpad"]
)

llm_chain = LLMChain(llm=ChatOpenAI(temperature=0.4,model_name="gpt-3.5-turbo-16k"), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

In [3]:
agent_chain("output Near Light")



> Entering new AgentExecutor chain...
Thought: I should output the content of the "Near Light" chapter from the Arknights story.
Action: output chapter content to html
Action Input: "Near Light"
Observation: success: write query to output.html
Thought:I have successfully outputted the content of the "Near Light" chapter to an HTML file.
Final Answer: The content of the "Near Light" chapter has been outputted to an HTML file.

> Finished chain.


{'input': 'output Near Light',
 'chat_history': '',
 'output': 'The content of the "Near Light" chapter has been outputted to an HTML file.'}

In [4]:
agent_chain("why was Maria kidnapped?")



> Entering new AgentExecutor chain...
Thought: I should answer the question by querying the vector database.
Action: answer story questions using vector db
Action Input: why was Maria kidnapped?
Observation: In the bar in Kazimierz, Maria and her aunt Zofia have an argument about Maria quitting the knight tournament. Zofia storms out, leaving Maria conflicted. (Stage: Bar in Kazimierz)

The Plastic Knight informs Maria that he has been replaced in the starting lineup for the Major tournament by a new knight. Maria participates in a knight racing bout and finishes in second place, bringing her closer to qualifying for the Major tournament. (Stage: Plastic Knight's visit to the bar)

The Flametail Knight, Sona, and the Ashlock Knight encounter an unknown individual who has been following them. They engage in a brief conversation before parting ways. (Stage: Encounter with unknown individual)

Maria has a tense confrontation with her uncle, Młynar, who urges her to quit the tournament a

{'input': 'why was Maria kidnapped?',
 'chat_history': 'Human: output Near Light\nAI: The content of the "Near Light" chapter has been outputted to an HTML file.',
 'output': 'Maria was kidnapped because of her involvement in the knight tournament and her refusal to quit despite pressure from her family and the corporations.'}

In [5]:
agent_chain.run("write a fan fiction of Nearl passionately kissing Vivana after their match")



> Entering new AgentExecutor chain...
Thought: I need to generate a fan fiction story based on the given prompt.
Action: fan_fiction
Action Input: Nearl passionately kisses Vivana after their match
Observation: The story continues with Maria Nearl, now the talk of the town after her surprising victory over the Left-hand Knight, Tytus Topola. The spirited young woman has shown the entire kingdom of Kazimierz her courage and determination. Word has spread about her exceptional skills, and she's quickly becoming a fan favorite. However, with every victory comes a new challenge.

A new competitor steps into the arena, a woman known as Vivana. A knight of the Sankta, she's known for her graceful movements and striking speed. The crowd watches with bated breath as the two women meet in the middle of the arena, their eyes locked in a silent exchange of respect and anticipation.

The match begins with a flurry of movement. Maria's attacks are powerful, but Vivana moves like water, dodging an

'Nearl passionately kisses Vivana after their match in a moment of celebration and mutual respect.'

In [7]:
#call local api for unrestricted language model
#use the first model to query and summarize the background first, 
context = prts_query.run("Viviana and Margaret Nearl")

In [34]:
#then pass it to the local model via api
res = prts_fiction.run_api(context, "\nBased on the context above, write an lenghty yuri fan fiction of Viviana and Margaret")
print(res)

 
Viviana gazed at the Radiant Knight in admiration as they sparred together on the training field. She had heard many rumors about knights like Margaret who were infected with the Infected Arts but still continued to fight alongside non-infected knights. Despite being surrounded by doubt and discrimination, Margaret remained steadfast in her commitment to upholding justice and protecting those in need. As she fought, Viviana couldn't help but feel drawn to this remarkable woman. Her strength, determination, and unwavering sense of duty reminded him so much of his own past as a knight.
As their sparring match came to an end, both women panted heavily from exertion. They had pushed each other to their limits during the intense training session. Viviana approached Margaret with caution, unsure how she would react to his admiration for her skills. But when he saw the smile on her face, he knew that he could trust her with his feelings. "Margaret," he began hesitantly, "I must confess some